In [1]:
import pandas as pd
import numpy as np
from pandas.tseries.holiday import USFederalHolidayCalendar
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 25)

In [13]:
data = pd.read_csv('../../data/yellow_trip_aggregated_dataset.csv')

data['date'] = pd.to_datetime(data['date'])
data = data.sort_values('date', ascending=True).reset_index(drop=True)

In [14]:
data

,date,total_trips,total_passenger_count,total_trip_distance,total_tip_amount,total_amount,total_trip_duration
0,2011-01-01 00:00:00,12216.0,19776.0,34806.70,7484.09,147092.61,9741493.0
1,2011-01-01 01:00:00,13383.0,22092.0,40697.40,8684.43,170672.78,11434865.0
2,2011-01-01 02:00:00,12045.0,19511.0,37706.80,8268.32,154991.00,9972823.0
3,2011-01-01 03:00:00,10587.0,16634.0,34502.10,7644.41,136943.20,8293949.0
4,2011-01-01 04:00:00,7840.0,11767.0,27280.20,5551.39,104966.40,5934715.0
...,...,...,...,...,...,...,...
104435,2022-11-30 19:00:00,7290.0,9474.0,25275.32,20767.27,160161.94,7531030.0
104436,2022-11-30 20:00:00,6897.0,8943.0,152140.46,20392.05,150317.96,6335849.0
104437,2022-11-30 21:00:00,7023.0,9170.0,25853.18,20724.28,151589.62,6148027.0
104438,2022-11-30 22:00:00,6380.0,8389.0,25999.45,19943.73,147300.51,6035016.0


In [15]:
# Check if the intervals are properly spaced into 1 hour
print('Time interval ok?', ((data['date'].diff().dt.seconds > 3600) | (data['date'].diff().dt.seconds < 3600)).sum() == 0)

# Check missing values
missing_values_count = data.isnull().sum()
display(missing_values_count)

missing_values_flag = data.isnull().sum(axis=1) > 0
display(data.loc[missing_values_flag])

# Fix missing values
missing_value_columns = missing_values_count[missing_values_count > 0].index.to_list()
forward_fill = data[missing_value_columns].fillna(method='ffill')
backward_fill = data[missing_value_columns].fillna(method='bfill')
fill_value = (forward_fill + backward_fill) / 2
data.loc[missing_values_flag, missing_value_columns] = fill_value.loc[missing_values_flag, missing_value_columns]

# Check missing values agian
display(data.loc[missing_values_flag])

Time interval ok? False


date                     0
total_trips              0
total_passenger_count    0
total_trip_distance      0
total_tip_amount         0
total_amount             0
total_trip_duration      0
dtype: int64

,date,total_trips,total_passenger_count,total_trip_distance,total_tip_amount,total_amount,total_trip_duration


,date,total_trips,total_passenger_count,total_trip_distance,total_tip_amount,total_amount,total_trip_duration


In [16]:
# Date features
data['month'] = data['date'].dt.month
data['week'] = data['date'].dt.isocalendar().week
data['day_of_week'] = data['date'].dt.dayofweek
data['day'] = data['date'].dt.day
data['hour'] = data['date'].dt.hour

# Determining if the day is a US federal holiday
cal = USFederalHolidayCalendar()
holidays = cal.holidays(start=data['date'].min() - pd.Timedelta(days=365), end=data['date'].max() + pd.Timedelta(days=365))
data['holiday'] = pd.to_datetime(data['date'].dt.strftime('%Y-%m-%d')).isin(holidays)


# Days since last holiday
data['days_since_last_holiday'] = pd.NaT
data.loc[data['holiday'], 'days_since_last_holiday'] = data.loc[data['holiday'], 'date'].values
data['days_since_last_holiday'] = data['days_since_last_holiday'].fillna(method='ffill').fillna(holidays[holidays <= data['date'].min()].max())
data['days_since_last_holiday'] = (data['date'] - data['days_since_last_holiday']).dt.total_seconds() / 3600 / 24

# Days to next holiday
data['days_until_next_holiday'] = pd.NaT
data.loc[data['holiday'], 'days_until_next_holiday'] = data.loc[data['holiday'], 'date'].values
data['days_until_next_holiday'] = data['days_until_next_holiday'].fillna(method='bfill').fillna(holidays[holidays >= data['date'].max()].min())
data['days_until_next_holiday'] = (data['days_until_next_holiday'] - data['date']).dt.total_seconds() / 3600 / 24

# Lagged features
base_columns = ['total_trips', 'total_passenger_count', 'total_trip_distance', 'total_tip_amount', 'total_amount', 'total_trip_duration']
lags_in_days = [1, 2, 3, 4, 5, 6, 7] #, 14, 21, 28, 64, 84] #12 weeks, 3 months

dfs = {}

for column in base_columns:
    for lag in lags_in_days:
        dfs['{}_in_last_{}_days'.format(column, lag)] = data[column].rolling(window=lag * 24 + 1).sum()
        dfs['{}_{}_days_ago'.format(column, lag)] = data[column].shift(lag * 24)

data = pd.concat([data, pd.DataFrame(dfs)], axis=1)


In [17]:
# Correct data for forecast horizon
forcast_horizon = 7

dataset = data.drop(columns = ['total_passenger_count', 'total_trip_distance', 'total_tip_amount', 'total_amount', 'total_trip_duration'])
dataset = dataset.rename(columns={'total_trips': 'demand', 'date':'timestamp'})

dataset.insert(2, 'inference_time', dataset['timestamp'])
dataset = dataset.iloc[max(lags_in_days)*24:,].reset_index(drop=True)

dataset[['timestamp', 'demand']] = dataset[['timestamp', 'demand']].shift(-forcast_horizon * 24)

dataset = dataset.loc[~dataset['timestamp'].isnull()]

In [18]:
dataset

,timestamp,demand,inference_time,month,week,day_of_week,day,hour,holiday,days_since_last_holiday,days_until_next_holiday,total_trips_in_last_1_days,...,total_trip_duration_in_last_2_days,total_trip_duration_2_days_ago,total_trip_duration_in_last_3_days,total_trip_duration_3_days_ago,total_trip_duration_in_last_4_days,total_trip_duration_4_days_ago,total_trip_duration_in_last_5_days,total_trip_duration_5_days_ago,total_trip_duration_in_last_6_days,total_trip_duration_6_days_ago,total_trip_duration_in_last_7_days,total_trip_duration_7_days_ago
0,2011-01-15 00:00:00,28409.0,2011-01-08 00:00:00,1,1,5,8,0,False,8.000000,9.000000,514439.0,...,653590616.0,7932138.0,950918012.0,6836187.0,1.230411e+09,5721764.0,1.479726e+09,6222412.0,1.586190e+09,5115966.0,1.715639e+09,9741493.0
1,2011-01-15 01:00:00,23681.0,2011-01-08 01:00:00,1,1,5,8,1,False,8.041667,8.958333,520833.0,...,659946374.0,4889501.0,958369721.0,4102893.0,1.238977e+09,3548101.0,1.487792e+09,4030499.0,1.595362e+09,4130277.0,1.720185e+09,11434865.0
2,2011-01-15 02:00:00,19960.0,2011-01-08 02:00:00,1,1,5,8,2,False,8.083333,8.916667,528837.0,...,666599172.0,3094844.0,965809127.0,2627485.0,1.246971e+09,2298391.0,1.495304e+09,2623408.0,1.602774e+09,3497604.0,1.720293e+09,9972823.0
3,2011-01-15 03:00:00,15359.0,2011-01-08 03:00:00,1,1,5,8,3,False,8.125000,8.875000,535868.0,...,672449849.0,2106676.0,972127163.0,1902728.0,1.253619e+09,1578918.0,1.501626e+09,1723150.0,1.608222e+09,2964597.0,1.719265e+09,8293949.0
4,2011-01-15 04:00:00,9962.0,2011-01-08 04:00:00,1,1,5,8,4,False,8.166667,8.833333,539672.0,...,675996913.0,1770265.0,975878175.0,1697241.0,1.257693e+09,1567214.0,1.505556e+09,1992104.0,1.610911e+09,2325073.0,1.716625e+09,5934715.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104099,2022-11-30 19:00:00,7290.0,2022-11-23 19:00:00,11,47,2,23,19,False,11.833333,0.208333,115380.0,...,256558208.0,5978580.0,367206722.0,4469438.0,4.638584e+08,7212390.0,5.669687e+08,6894392.0,6.792142e+08,6913530.0,7.945942e+08,7046728.0
104100,2022-11-30 20:00:00,6897.0,2022-11-23 20:00:00,11,47,2,23,20,False,11.875000,0.166667,113212.0,...,255394949.0,4720204.0,367552605.0,3341538.0,4.614613e+08,5902951.0,5.648896e+08,5809434.0,6.771160e+08,6066074.0,7.923628e+08,5792558.0
104101,2022-11-30 21:00:00,7023.0,2022-11-23 21:00:00,11,47,2,23,21,False,11.916667,0.125000,112140.0,...,255746612.0,4186484.0,369282934.0,2746214.0,4.606302e+08,5226713.0,5.641521e+08,5584354.0,6.761218e+08,6422138.0,7.916421e+08,5568959.0
104102,2022-11-30 22:00:00,6380.0,2022-11-23 22:00:00,11,47,2,23,22,False,11.958333,0.083333,110767.0,...,255680661.0,4298715.0,370657253.0,2680095.0,4.595241e+08,5533508.0,5.626882e+08,5812714.0,6.738202e+08,5710735.0,7.901937e+08,5193649.0


In [12]:
dataset.to_csv('../../data/yellow_trip_demand_forecasting_dataset.csv', index=False)